<a href="https://colab.research.google.com/github/chottokun/ModernBERT_NER_ja/blob/main/modernBERT_JA_NER_Sudachi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ModernBERT NER の整理



In [1]:
# ModernBERT ベースモデル・HFレポジトリ名設定

# ベースモデルとなるモデル　hugging
model_name = "cl-nagoya/ruri-v3-130m"

# Hugging Faceユーザー名または組織名 (例: "your-username" または "your-org")
# ログインしているユーザーのリポジトリに作成する場合、ユーザー名または組織名は不要です。
# repo_id = "your-username/japanese-ner-wikipedia-sudachi" # ユーザー名を指定する場合
repo_id = "Chottokun/ruri-v3-pt-130m_ner_wikipedia" # ログインユーザーのリポジトリに作成する場合

# 学習条件
EARLY_STOP = True # False に設定すると Early Stopping は無効になります
NUM_TRAIN_EPOCHS = 50 # Early Stoppingを使用する場合、num_train_epochsは最大エポック数となる


## Hugging Face Hub へのログイン

### Subtask:
Hugging Face Hub に HF-TOKEN を使用してログインします。

**Reasoning**:
Log in to the Hugging Face Hub programmatically using the HF-TOKEN stored as a Colab secret for authentication.

In [2]:
from huggingface_hub import notebook_login
from google.colab import userdata

# HF-TOKEN を Colab の Secrets から取得してログイン
try:
    hf_token = userdata.get('HF_TOKEN')
    if hf_token:
        # notebook_login() # これによりトークン入力プロンプトが表示されるか、既にログイン済みならスキップされる
        # Alternatively, you can directly login with the token if notebook_login is not interactive enough
        from huggingface_hub import login
        login(token=hf_token)
    else:
        print("Colab Secrets に 'HF_TOKEN' が設定されていません。設定してください。")
except Exception as e:
    print(f"Hugging Face Hub ログイン中にエラーが発生しました: {e}")

## ライブラリのインストール

### Subtask:
必要なライブラリ（`transformers`, `datasets`, `SudachiPy`, `SudachiDict_core`, `spacy-alignments`, `seqeval`, `evaluate`）をインストールします。

In [3]:
!pip install -U -q transformers datasets SudachiPy SudachiDict_core spacy-alignments seqeval evaluate

## データセットとトークナイザーの準備

### Subtask:
データセットをロードし、モデルのトークナイザー、Sudachiトークナイザー、およびラベルのマッピングを準備します。

In [4]:
from transformers import AutoTokenizer
from datasets import load_dataset
from sudachipy import Dictionary, SplitMode

# モデルとトークナイザーの準備
# model_name = "cl-nagoya/ruri-v3-130m"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Sudachi辞書の準備
dict_obj = Dictionary()
tokenizer_obj = dict_obj.create()

# カスタムSudachiトークナイザー関数の定義
def sudachi_tokenizer(text):
    # A mode for short units
    tokens = [m.surface() for m in tokenizer_obj.tokenize(text, SplitMode.A)]
    return tokens

# データセットの準備
dataset = load_dataset("stockmark/ner-wikipedia-dataset")

# ラベルのマッピング
label_list = ["O", "B-人名", "I-人名", "B-法人名", "I-法人名", "B-政治的組織名", "I-政治的組織名",
              "B-その他の組織名", "I-その他の組織名", "B-地名", "I-地名", "B-施設名", "I-施設名",
              "B-製品名", "I-製品名", "B-イベント名", "I-イベント名"]
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}

print("データセット、トークナイザー、ラベルマッピングの準備が完了しました。")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/1.83M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

データセット、トークナイザー、ラベルマッピングの準備が完了しました。


## データ前処理とラベルアライメント

### Subtask:
Sudachiで分かち書きを行い、Hugging Faceトークナイザーでサブワード化し、`spacy-alignments`を用いてラベルをアライメントします。

In [5]:
import spacy_alignments as tokenizations # Import spacy_alignments

def tokenize_and_align_labels_sudachi(examples):
    all_input_ids = []
    all_attention_mask = []
    all_labels = []

    for i, text in enumerate(examples["text"]):
        entities = examples["entities"][i]
        # 1. Sudachiで分かち書きを行う
        words = sudachi_tokenizer(text)

        # 2. 分かち書きした単語列をHugging Faceトークナイザーに入力
        #    is_split_into_words=True を使用し、return_offsets_mapping=True でオフセット情報を取得
        tokenized_inputs = tokenizer(
            words,
            is_split_into_words=True,
            truncation=True,
            return_offsets_mapping=True,
            return_attention_mask=True,
            add_special_tokens=True # Special tokens are added here
        )

        input_ids = tokenized_inputs["input_ids"]
        attention_mask = tokenized_inputs["attention_mask"]
        offset_mapping = tokenized_inputs["offset_mapping"]

        # 3. spacy_alignments を使用して単語とサブワードのアライメントを取得
        #    Hugging Face tokenizer converts words to subwords, so we need to convert token_ids back to tokens for alignment
        #    Exclude special tokens from subwords for alignment when creating subwords_for_alignment
        subwords_for_alignment = tokenizer.convert_ids_to_tokens(tokenizer(words, is_split_into_words=True, add_special_tokens=False).input_ids)


        words2subwords, subwords2words = tokenizations.get_alignments(words, subwords_for_alignment)


        # 4. エンティティラベルをサブワードにアライメント
        aligned_labels = [-100] * len(input_ids)

        # Create word-level labels first based on original text and entities
        word_labels = ["O"] * len(words)
        # Calculate character spans for each word in the original text
        word_char_spans = []
        current_char_index = 0
        for word_text in words:
            start_index = text.find(word_text, current_char_index)
            if start_index != -1:
                end_index = start_index + len(word_text)
                word_char_spans.append((start_index, end_index))
                current_char_index = end_index
            else:
                 word_char_spans.append((-1, -1)) # Should not happen with correct tokenization

        # Assign B- or I- labels at the word level
        for ent in entities:
             ent_start_char, ent_end_char = ent["span"]
             ent_type = ent["type"]

             for word_index, (word_char_start, word_char_end) in enumerate(word_char_spans):
                  if word_char_start != -1 and max(word_char_start, ent_start_char) < min(word_char_end, ent_end_char):
                       # Word overlaps with an entity
                       if word_char_start >= ent_start_char: # If the word starts at or after the entity start
                            is_entity_start = False
                            # Check if the previous word is part of the same entity
                            if word_index == 0:
                                is_entity_start = True
                            else:
                                prev_word_char_start, prev_word_char_end = word_char_spans[word_index - 1]
                                prev_word_in_same_entity = False
                                if prev_word_char_start != -1:
                                     for prev_ent in entities:
                                          prev_ent_start, prev_ent_end = prev_ent["span"]
                                          prev_ent_type = prev_ent["type"]
                                          # Check if the previous word's character span overlaps with the same entity type
                                          if max(prev_word_char_start, prev_ent_start) < min(prev_word_char_end, prev_ent_end) and prev_ent_type == ent_type:
                                                prev_word_in_same_entity = True
                                                break
                                if not prev_word_in_same_entity:
                                     is_entity_start = True

                            word_labels[word_index] = "B-" + ent_type if is_entity_start else "I-" + ent_type


        # Transfer word-level labels to subword-level labels using alignments
        for subword_index in range(len(input_ids)):
            if tokenizer.convert_ids_to_tokens(input_ids[subword_index]) in tokenizer.all_special_tokens:
                 aligned_labels[subword_index] = -100
            else:
                 # Get the word index(es) aligned to this subword (adjusting for special tokens)
                 # The alignment from spacy_alignments is based on subwords_for_alignment (without special tokens)
                 # We need to map the current subword_index (from input_ids with special tokens)
                 # to the corresponding index in subwords_for_alignment.
                 # Assuming special tokens are only at the beginning ([CLS]) and end ([SEP]),
                 # the subword at index `k` in input_ids (where k > 0 and k < len(input_ids) - 1)
                 # corresponds to the subword at index `k-1` in subwords_for_alignment.
                 subword_alignment_index = subword_index - 1 # Adjust for [CLS] token at the beginning

                 if subword_alignment_index >= 0 and subword_alignment_index < len(subwords2words):
                      aligned_word_indices = subwords2words[subword_alignment_index]

                      if aligned_word_indices:
                           # Take the label of the first aligned word
                           first_aligned_word_index = aligned_word_indices[0]
                           if first_aligned_word_index < len(word_labels):
                                word_level_label = word_labels[first_aligned_word_index]

                                if word_level_label != "O":
                                     # If this subword is the first subword of the first aligned word, assign B-
                                     # Otherwise, assign I-
                                     is_first_subword_of_word = (first_aligned_word_index < len(words2subwords)) and (subword_alignment_index in words2subwords[first_aligned_word_index]) and (words2subwords[first_aligned_word_index][0] == subword_alignment_index)


                                     if is_first_subword_of_word:
                                          aligned_labels[subword_index] = label2id[word_level_label]
                                     else:
                                          # Ensure I- label matches the B- label type
                                          if word_level_label.startswith("B-"):
                                               aligned_labels[subword_index] = label2id["I-" + word_level_label.split("-")[1]]
                                          else:
                                               aligned_labels[subword_index] = label2id[word_level_label] # It's already an I- label at word level
                                else:
                                     aligned_labels[subword_index] = label2id["O"]
                      else:
                           # Subword aligned to no word (should not happen with is_split_into_words=True and proper alignment)
                           aligned_labels[subword_index] = label2id["O"] # Default to O
                 else:
                      # This case handles the last special token ([SEP]) and any potential padding tokens
                      aligned_labels[subword_index] = -100


        all_input_ids.append(input_ids)
        all_attention_mask.append(attention_mask)
        all_labels.append(aligned_labels)


    return {
        "input_ids": all_input_ids,
        "attention_mask": all_attention_mask,
        "labels": all_labels
    }


# データセットに前処理を適用 (分割前にmapを適用)
tokenized_datasets_sudachi = dataset.map(
    tokenize_and_align_labels_sudachi,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

# "train" 分割から 10% を検証用として作成 (map処理後に分割)
split_datasets_sudachi = tokenized_datasets_sudachi["train"].train_test_split(test_size=0.1)

print("データ前処理とラベルアライメントが完了しました。")

# 確認のために最初の数件を表示
df_sudachi = split_datasets_sudachi["train"].select(range(5)).to_pandas()
display(df_sudachi)

Parameter 'function'=<function tokenize_and_align_labels_sudachi at 0x7b0d1bcccf40> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/5343 [00:00<?, ? examples/s]

データ前処理とラベルアライメントが完了しました。


,input_ids,attention_mask,labels
0,"[1, 4560, 6066, 400, 577, 1231, 302, 275, 9457...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 1, 0, 13, 14, 14, ..."
1,"[1, 513, 275, 24022, 9981, 312, 11326, 49914, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, ..."
2,"[1, 1277, 320, 22146, 811, 826, 311, 3780, 469...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 9, 0, 0, 0, 0, 3, 4, 0, 0, ..."
3,"[1, 18971, 75033, 13018, 302, 275, 277, 299, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[1, 13744, 756, 345, 32281, 24692, 291, 8024, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## トレーニングの実行

### Subtask:
更新したデータセットでモデルのトレーニングを実行します。

In [6]:
import torch
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, get_linear_schedule_with_warmup, DataCollatorForTokenClassification
import numpy as np
import evaluate
from transformers.trainer_callback import EarlyStoppingCallback

# Early Stopping を有効にするかのフラグ
EARLY_STOP = True # False に設定すると Early Stopping は無効になります

# モデルの準備
# model_name = "sbintuitions/modernbert-ja-130m"
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))

# 評価指標の準備
metric = evaluate.load("seqeval")

# 評価指標の計算関数
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

# トレーニングの設定
training_args = TrainingArguments(
    output_dir="./results_sudachi",
    eval_strategy="epoch", # Early Stoppingにはevaluation_strategyが"epoch"または"steps"である必要がある
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs_sudachi",
    num_train_epochs=NUM_TRAIN_EPOCHS, # Early Stoppingを使用する場合、num_train_epochsは最大エポック数となる
    weight_decay=0.01,
    load_best_model_at_end=True, # Early Stoppingにはload_best_model_at_endがTrueである必要がある
    save_strategy="epoch",
    report_to="none",
    metric_for_best_model="f1", # 監視する評価指標 (compute_metricsのキーと一致させる)
    greater_is_better=True, # 監視する評価指標が大きい方が良いか (f1は大きい方が良い)
)

# オプティマイザと学習率スケジューラの設定
optimizer = torch.optim.AdamW(model.parameters(), lr=training_args.learning_rate) # Use learning_rate from training_args
num_training_steps = len(split_datasets_sudachi["train"]) // training_args.per_device_train_batch_size * training_args.num_train_epochs
num_warmup_steps = int(training_args.warmup_ratio * num_training_steps) if training_args.warmup_ratio is not None else training_args.warmup_steps

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

# データコレーターの作成
data_collator = DataCollatorForTokenClassification(tokenizer)

# Early Stopping コールバックの設定 (条件付きで追加)
callbacks = []
if EARLY_STOP:
    early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=3, # 監視指標がこのエポック/ステップ数改善しない場合に停止
        early_stopping_threshold=0.0, # 改善とみなす最小の変化量 (0.0でわずかな変化でも改善とみなす)
    )
    callbacks.append(early_stopping_callback)


# Trainerの初期化と学習の実行
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_datasets_sudachi["train"],
    eval_dataset=split_datasets_sudachi["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer, scheduler),
    callbacks=callbacks, # Callbacks リストを渡す
)

# トレーニングの開始
trainer.train()

# モデルの保存
trainer.save_model("./japanese_ner_wikipedia_sudachi")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/529M [00:00<?, ?B/s]

Some weights of ModernBertForTokenClassification were not initialized from the model checkpoint at cl-nagoya/ruri-v3-130m and are newly initialized: ['classifier.bias', 'classifier.weight', 'head.dense.weight', 'head.norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2833509131.py:77: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.125187,0.790366,0.834527,0.811847,0.967524
2,0.134900,0.133698,0.786685,0.829513,0.807531,0.964913
3,0.134900,0.158194,0.813172,0.866762,0.839112,0.965863
4,0.039500,0.148432,0.816092,0.864613,0.839652,0.966860
5,0.018600,0.163200,0.842958,0.857450,0.850142,0.970088
6,0.018600,0.170651,0.844720,0.876791,0.860457,0.970421
7,0.009100,0.205689,0.824131,0.866046,0.844569,0.968901
8,0.009100,0.194825,0.840448,0.860315,0.850265,0.970895
9,0.006500,0.203875,0.833217,0.855301,0.844115,0.970326


## モデルの評価 (詳細)

### Subtask:
トレーニング済みのモデルをテストデータセットで評価し、詳細な評価指標を表示します。

In [7]:
# トレーニング済みモデルでの評価を実行
evaluation_results = trainer.evaluate(split_datasets_sudachi["test"])

# 評価結果を表示
print("--- 詳細な評価結果 ---")
import pprint # Add import statement
pprint.pprint(evaluation_results)

--- 詳細な評価結果 ---
{'epoch': 9.0,
 'eval_accuracy': 0.9704206628050518,
 'eval_f1': 0.860456942003515,
 'eval_loss': 0.1706513613462448,
 'eval_precision': 0.84472049689441,
 'eval_recall': 0.8767908309455588,
 'eval_runtime': 3.057,
 'eval_samples_per_second': 175.01,
 'eval_steps_per_second': 11.122}


## サンプル予測結果の確認

### Subtask:
テストデータセットからいくつかのサンプルを選び、モデルの予測結果を元の文章、正解ラベルと比較して表示します。

In [8]:
# 実際の予測結果を確認する
predicted_labels = []

# テストデータセットから10個をサンプリング
for i in range(0, 10):
    # テストデータセットからサンプルを1つ選択
    sample = split_datasets_sudachi["test"][i] # Use split_datasets_sudachi

    # Trainerのpredictメソッドはデータセットまたはデータセットのリストを受け取る
    # 単一のサンプルに対して予測を行う場合は、データセット形式に合わせる必要がある
    # または、pipelineを使用する方が単一テキストの予測には適している

    # trainer.predict([sample]) の代わりに pipeline を使用して手軽に予測を行う
    # ただし、pipelineはモデルのロードが必要。ここではtrainerオブジェクトが使えるのでtrainer.predictを使う方向で調整
    # trainer.predictはBatchConverterを内部で使用するため、単一サンプルをリストで渡すのが適切
    predictions = trainer.predict([sample])


    # 予測結果を解釈
    predicted_label_ids = predictions.predictions.argmax(-1)[0]

    # special tokenを除外してラベル名に変換
    # sample["labels"] も特殊トークンは-100になっている前提
    true_predictions = [
        label_list[p_id] for p_id, l_id in zip(predicted_label_ids, sample["labels"]) if l_id != -100
    ]
    true_labels = [
        label_list[l_id] for l_id in sample["labels"] if l_id != -100
    ]

    # トークンを取得 (特殊トークンを除外)
    tokens = tokenizer.convert_ids_to_tokens(sample['input_ids'])
    # アライメントされたラベルに対応するトークンのみを取得するため、-100でないラベルに対応するトークンを選ぶ
    aligned_tokens = [token for token, l_id in zip(tokens, sample["labels"]) if l_id != -100]


    # 元の文章と予測結果を出力
    print("元の文章トークン:", ' '.join(aligned_tokens))
    print("予測ラベル:", true_predictions)  # special tokenを除外した予測結果を出力
    print("正解ラベル:", true_labels) # special tokenを除外した正解ラベルを出力
    # 正誤判定はラベル名で行う
    print("正誤:", ["○" if p == l else "×" for p, l in zip(true_predictions, true_labels)])
    print("---")

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


元の文章トークン: 天皇 賞 で は 6 番 手 から レース を 進め た が 、 直線 で 前方 へ 進出 する ため の 進路 を 確保 する こと が でき な かっ た ため に 加速 する の が 遅れ 、 先 に 抜け 出し た スーパー クリー ク を 交わ す こと が でき ず 2 着 に 敗 れ た 。
予測ラベル: ['B-イベント名', 'I-イベント名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-イベント名', 'I-製品名', 'I-製品名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
正解ラベル: ['B-イベント名', 'I-イベント名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
正誤: ['○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○

元の文章トークン: こう し た 理由 から 固定 し た ファン 層 の 獲得 が でき ず 、 運営 面 で も 大口 の スポンサー を 確保 でき な かっ た ため 、 大塚 アスレチック ス の 大塚 幸 之助 以外 の 経営 者 は 興行 を 続ける 力 を 失 っ て い た 。
予測ラベル: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-法人名', 'I-法人名', 'I-法人名', 'O', 'B-人名', 'I-人名', 'I-人名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
正解ラベル: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-法人名', 'I-法人名', 'I-法人名', 'O', 'B-人名', 'I-人名', 'I-人名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
正誤: ['○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○'

元の文章トークン: 1 9 5 8 年 に サンフランシスコ ・ ジャイアンツ に フリー エージェント で 入団 、 1 9 6 6 年 に クリー ブランド ・ インディアン ズ で メジャー リーグ に 初 出場 し 1 9 6 8 年 まで 在籍 、 1 9 6 9 年 に シアトル ・ パイ ロ ッツ に 移籍 、 1 9 7 1 年 に は 日本 の 西鉄 ライオンズ で プレイ し た 。
予測ラベル: ['O', 'O', 'O', 'O', 'O', 'O', 'B-その他の組織名', 'I-その他の組織名', 'I-その他の組織名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-その他の組織名', 'I-その他の組織名', 'I-その他の組織名', 'I-その他の組織名', 'I-その他の組織名', 'O', 'B-その他の組織名', 'I-その他の組織名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-その他の組織名', 'I-その他の組織名', 'I-その他の組織名', 'I-その他の組織名', 'I-その他の組織名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-地名', 'O', 'B-その他の組織名', 'I-その他の組織名', 'O', 'O', 'O', 'O', 'O']
正解ラベル: ['O', 'O', 'O', 'O', 'O', 'O', 'B-その他の組織名', 'I-その他の組織名', 'I-その他の組織名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-その他の組織名', 'I-その他の組織名', 'I-その他の組織名', 'I-その他の組織名', 'I-その他の組織名', 'O', 'B-その他の組織名', 'I-その他の組織名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

元の文章トークン: クリントン 政権 期 に 膨大 な 数 の 公的 文書 が 機密 解除 さ れ た 現在 で は 、 ニク ソン 政権 に よる 金融 封鎖 その 他 の 経済 <0xE6> <0x94> <0xAA> 乱 工作 さえ な け れ ば ア ジェン デ 政権 の 経済 運営 は 順調 に 行 っ て い た 、 と する 説 が 有力 で ある 。
予測ラベル: ['B-政治的組織名', 'I-政治的組織名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-政治的組織名', 'I-政治的組織名', 'I-政治的組織名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-政治的組織名', 'I-政治的組織名', 'I-政治的組織名', 'I-政治的組織名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
正解ラベル: ['B-政治的組織名', 'I-政治的組織名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-政治的組織名', 'I-政治的組織名', 'I-政治的組織名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-政治的組織名', 'I-政治的組織名', 'I-政治的組織名', 'I-政治的組織名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
正誤: ['○', '○', '○', '○'

...

元の文章トークン: ビジネス モデル に 類似 の 概念 と し て 、 伊丹 敬 之 、 加護 野 忠 男 が キー 概念 と し て いる 「 ビジネス システム 」 が ある 。
予測ラベル: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-人名', 'I-人名', 'I-人名', 'O', 'B-人名', 'I-人名', 'I-人名', 'I-人名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
正解ラベル: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-人名', 'I-人名', 'I-人名', 'O', 'B-人名', 'I-人名', 'I-人名', 'I-人名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
正誤: ['○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○']
---


元の文章トークン: この 時 から 徐 <0xE9> <0xA9> <0x8E> は ライジング スター と し て 注目 を 集め て いる 。
予測ラベル: ['O', 'O', 'O', 'B-人名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
正解ラベル: ['O', 'O', 'O', 'B-人名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
正誤: ['○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○']
---


元の文章トークン: 東 日本 大 震災 の 復興 支援 を 目的 と し て 2 0 1 3 年 に 始 まっ た 。
予測ラベル: ['B-イベント名', 'I-イベント名', 'I-イベント名', 'I-イベント名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
正解ラベル: ['B-イベント名', 'I-イベント名', 'I-イベント名', 'I-イベント名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
正誤: ['○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○']
---


元の文章トークン: 官 房 長官 秘書 ・ 福 本 邦 雄 の 依頼 を 受け 、 長官 官 舎 で ペン を 執 っ た 。
予測ラベル: ['O', 'O', 'O', 'O', 'O', 'B-人名', 'I-人名', 'I-人名', 'I-人名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
正解ラベル: ['O', 'O', 'O', 'O', 'O', 'B-人名', 'I-人名', 'I-人名', 'I-人名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
正誤: ['○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○']
---


元の文章トークン: 園内 に 設け られ た 動物 ふれあい 広場 で は 、 動物 と 直接 触れる こと が 出来 た 。
予測ラベル: ['O', 'O', 'O', 'O', 'O', 'B-施設名', 'I-施設名', 'I-施設名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
正解ラベル: ['O', 'O', 'O', 'O', 'O', 'B-施設名', 'I-施設名', 'I-施設名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
正誤: ['○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○', '○']
---


## Hugging Face Hub にモデルをプッシュ

### Subtask:
Hugging Face Hub に新しいリポジトリを作成し、トレーニング済みのモデルファイルをアップロードします。

In [9]:
from huggingface_hub import create_repo, HfApi
import os

# Hugging Faceユーザー名または組織名 (例: "your-username" または "your-org")
# ログインしているユーザーのリポジトリに作成する場合、ユーザー名または組織名は不要です。
# repo_id = "your-username/japanese-ner-wikipedia-sudachi" # ユーザー名を指定する場合
# repo_id = "Chottokun/ruri-v3-pt-130m_ner_wikipedia" # ログインユーザーのリポジトリに作成する場合

# リポジトリを作成
try:
    create_repo(repo_id, repo_type="model", exist_ok=True)
    print(f"リポジトリ '{repo_id}' が作成または既に存在します。")
except Exception as e:
    print(f"リポジトリの作成中にエラーが発生しました: {e}")


# トレーニング済みのモデルディレクトリ
local_model_path = "./japanese_ner_wikipedia"

# モデルファイルをHugging Face Hubにプッシュ
# Trainer オブジェクトが push_to_hub メソッドを持っている場合
try:
    if 'trainer' in locals() and hasattr(trainer, 'push_to_hub'):
        print(f"'{local_model_path}' のモデルファイルを '{repo_id}' にプッシュします...")
        trainer.push_to_hub(repo_id)
        print("モデルのプッシュが完了しました。")
    else:
        print("Trainer オブジェクトが見つからないか、push_to_hub メソッドがありません。")
        print("代わりに HfApi を使用してファイルをアップロードします。")

        # HfApi を使用してファイルをアップロードする場合
        api = HfApi()
        # アップロードするファイルリスト (必要に応じて調整)
        files_to_upload = [
            os.path.join(local_model_path, "config.json"),
            os.path.join(local_model_path, "pytorch_model.bin"),
            os.path.join(local_model_path, "training_args.bin"),
            # Add other necessary files like tokenizer files if they were saved to local_model_path
            os.path.join(local_model_path, "tokenizer.json"),
            os.path.join(local_model_path, "special_tokens_map.json"),
            os.path.join(local_model_path, "tokenizer_config.json"),
            os.path.join(local_model_path, "vocab.txt"), # If using a vocab.txt based tokenizer
            os.path.join(local_model_path, "spm.model"), # If using a SentencePiece model
        ]

        for file_path in files_to_upload:
            if os.path.exists(file_path):
                try:
                    api.upload_file(
                        path_or_fileobj=file_path,
                        path_in_repo=os.path.basename(file_path),
                        repo_id=repo_id,
                        repo_type="model",
                    )
                    print(f"'{os.path.basename(file_path)}' をアップロードしました。")
                except Exception as e:
                    print(f"ファイルのアップロード中にエラーが発生しました ({os.path.basename(file_path)}): {e}")
            else:
                print(f"ファイル '{file_path}' が見つかりませんでした。スキップします。")


except Exception as e:
    print(f"モデルのプッシュ中にエラーが発生しました: {e}")

リポジトリ 'Chottokun/ruri-v3-pt-130m_ner_wikipedia' が作成または既に存在します。
'./japanese_ner_wikipedia' のモデルファイルを 'Chottokun/ruri-v3-pt-130m_ner_wikipedia' にプッシュします...


model.safetensors:   0%|          | 0.00/530M [00:00<?, ?B/s]

モデルのプッシュが完了しました。
